In [1]:
!pip install pandas matplotlib ipywidgets

In [2]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
import matplotlib
from ipywidgets import interactive_output
import ipywidgets as widgets
from IPython.core.display import display

In [3]:
matplotlib.rcParams['axes.labelpad'] = 20


# Import dataset
df = pd.read_csv("cexdataweb.csv")
employed = df.refhpw > 0.  # Only keep data with positive work hours per week

In [4]:
def plot(age, max_cons, max_hour):
    # Data wrangling
    df_by_age = df[(df.refage == age) & employed]  # Condition on age and positive works hours per week
    hourly_salary = df_by_age.refwp / df_by_age.refhpw  # Compute hourly salary as measure of productivity
    cond_max_cons = df_by_age.texp1 < max_cons  # Condition: total expenditure < max_cons
    cond_max_hour = hourly_salary < max_hour  # Condition: max hourly salary < max_hour
    cond_max = (cond_max_cons) & (cond_max_hour)  # Combine conditions
    
    # Apply conditions to data
    hourly_salary = hourly_salary[cond_max] 
    cons = df_by_age.texp1[cond_max]

    # Create histogram
    nb_obs = hourly_salary.size
    hist, xedges, yedges = np.histogram2d(cons, hourly_salary, bins=20)
    hist = hist / nb_obs

    X, Y = np.meshgrid((yedges[:-1] + yedges[1:]) / 2., (xedges[:-1] + xedges[1:]) / 2.)

    # Create plot
    fig = plt.figure(figsize=(14, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    ax.plot_wireframe(X, Y, hist, color='b')

    ax.set_xlabel(r'hourly wage')
    ax.set_ylabel(r'total expenditure')
    ax.set_zlabel(r'probability mass for age %i' % age)
    ax.set_title(r'number of observations %i' % nb_obs)
    
    plt.show();
    

age = widgets.IntSlider(min=df.refage.min(), max=df.refage.max(), step=1, value=25, description='age')
max_cons = widgets.FloatSlider(min=0., max=40000, step=100., value=20000, description='max_cons')
max_hour = widgets.FloatSlider(min=0., max=200, step=10., value=100, description='max_hour')

ui = widgets.HBox([age, max_cons, max_hour])
    
controls = {'age': age,
           'max_cons': max_cons,
           'max_hour': max_hour}

hh = interactive_output(plot, controls)

display(ui, hh)

Output()

In [5]:
def plot(age, max_wea, max_hour):
    df_by_age = df[(df.refage == age) & employed]  # Condition on age and positive works hours per week
    hourly_salary = df_by_age.refwp / df_by_age.refhpw  # Compute hourly salary as measure of productivity
    total_wealth = df_by_age.finwea + df_by_age.propval  # Compute total wealth < max_cons
    cond_max_wea = (0. < total_wealth) &  (total_wealth < max_wea)  # Condition: 0. < total wealth < max_cons
    cond_max_hour = hourly_salary < max_hour  # Condition: max hourly salary < max_hour
    cond_max = (cond_max_wea) & (cond_max_hour)  # Combine conditions
    
    # Apply conditions to data
    hourly_salary = hourly_salary[cond_max]    
    total_wealth = total_wealth[cond_max]

    # Create histogram
    nb_obs = hourly_salary.size
    hist, xedges, yedges = np.histogram2d(total_wealth, hourly_salary, bins=20)
    hist = hist / nb_obs

    X, Y = np.meshgrid((yedges[:-1] + yedges[1:]) / 2., (xedges[:-1] + xedges[1:]) / 2.)
    
    # Create plot
    fig = plt.figure(figsize=(14, 10))
    ax = fig.add_subplot(111, projection='3d')

    ax.plot_wireframe(X, Y, hist, color='b')

    ax.set_xlabel(r'hourly wage')
    ax.set_ylabel(r'financial wealth')
    ax.set_zlabel(r'probability mass for age %i' % age)
    ax.set_title(r'number of observations %i' % nb_obs)
    
    plt.show();
    

age = widgets.IntSlider(min=df.refage.min(), max=df.refage.max(), step=1, value=25, description='age')
max_wea = widgets.FloatSlider(min=0., max=1_000_000., step=100., value=500000., description='max_wea')
max_hour = widgets.FloatSlider(min=0., max=200, step=10, value=100., description='max_hour')

ui = widgets.HBox([age, max_wea, max_hour])
    
controls = {'age': age,
           'max_wea': max_wea,
           'max_hour': max_hour}

hh = interactive_output(plot, controls)

display(ui, hh)

Output()